# Exercise *02*: `Join`

## Create a connection to the database using the library *sqlite3*:

In [1]:
import sqlite3

from sqlite3 import Connection

In [2]:
db_path: str = "../../data/checking_logs.sqlite"

In [3]:
conn: Connection = sqlite3.connect(db_path)

## Create a new table *datamart* in the database by joining the tables *pageviews* and *checker* using only one query:

* The table should have the following columns: *uid*, *labname*, *first_commit_ts*, *first_view_ts*:
* *first_commit_ts* is just a new name of the column *timestamp* from the *checker* table, it shows the first commit from a particular lab and from a particular user:
* *first_view_ts* is the first visit of a user to the table *pageviews*, timestamp when a user visited the *newsfeed*:
* `status = ’ready’` should still be a filter:
* `numTrials = 1` should still be a filter:
* labnames should still be from the list: `’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’`:
* The table should contain only the users (`uids with user_*`) and not the admins:
* *first_commit_ts* and *first_view_ts* should be parsed as *datetime64[ns]*.

In [4]:
import pandas as pd

from sqlite3 import Cursor
from pandas import Timestamp, DataFrame

In [5]:
sql_subquery: str = """
                    SELECT
                        checker.uid AS uid,
                        checker.labname AS labname,
                        MIN(checker.timestamp) AS first_commit_ts,
                        MIN(pageviews.datetime) AS first_view_ts
                    FROM
                        checker
                    FULL OUTER JOIN
                        pageviews ON checker.uid = pageviews.uid
                    WHERE
                        checker.status = 'ready'
                        AND checker.numTrials = 1
                        AND checker.uid LIKE 'user_%'
                        AND checker.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
                    GROUP BY
                        checker.uid,
                        checker.labname
                    """

In [6]:
sql_query: str = f"""
                  CREATE TABLE datamart AS
                      {sql_subquery};
                  """

In [7]:
curs: Cursor = conn.cursor()

In [8]:
curs.execute(sql_query)
conn.commit()

In [9]:
# sql_query: str = f"DROP TABLE datamart;"

In [10]:
# curs.execute(sql_query)
# conn.commit()

## Using *Pandas* methods, create two dataframes: *test* and *control*:

* *test* should have the users that have the values in *first_view_ts*:
* *control* should have the users that have missing values in *first_view_ts*:
* Replace the missing values in the *control* with the average *first_view_ts* of the *test* users, we will use this value for the future analysis:
* Save both tables into the database, you will use them in the next exercises.

In [11]:
sql_query: str = f"""
                  SELECT 
                      *
                  FROM 
                      datamart;
                  """

In [12]:
datamart: DataFrame = pd.io.sql.read_sql(
    sql_query,
    conn
)

In [13]:
datamart["first_view_ts"] = pd.to_datetime(datamart["first_view_ts"])
datamart["first_commit_ts"] = pd.to_datetime(datamart["first_commit_ts"])

In [14]:
test: DataFrame = datamart[datamart["first_view_ts"].notna()]
control: DataFrame = datamart[datamart["first_view_ts"].isna()]

In [15]:
avg_first_view_ts: Timestamp = test["first_view_ts"].mean()

In [16]:
control.loc[
    :,
    "first_view_ts",
] = control["first_view_ts"].fillna(avg_first_view_ts)

In [17]:
control.to_sql(
    "control",
    conn,
    if_exists="replace",
    index=False
)

test.to_sql(
    "test",
    conn,
    if_exists="replace",
    index=False
)

59

In [18]:
# test

In [19]:
# control

## Close the connection:

In [20]:
conn.close()